In [ ]:
import torch

def print_gpu_memory(prefix=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"{prefix} Memory Allocated: {allocated:.2f} MB")
        print(f"{prefix} Memory Reserved: {reserved:.2f} MB")
    else:
        print("CUDA is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 

print_gpu_memory()

In [ ]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))

import src.data.preprocess_data as data
import src.training.train_model as train
import src.data.view as view
import src.models.unets as unets
import src.models.hrnets as hrnets
import src.training.post_processing as post

from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
model_name = 'UNetSmall-256-type-CE-diceW.pth'
model_name = 'UNetSmall-256-type-crf-CEW.pth'
#model_name = 'UNet-256-type-CE.pth'
#model_name = 'UNetConvNext-224-type-CEW.pth'
#model_name = 'HRNetW32-512-type-CEW.pth'
patch_size = int(model_name.split('-')[1])
weighted = 'W.pth' in model_name
if model_name.split('-')[2]=='type':
    num_classes = 5
if model_name.split('-')[2]=='binary':
    num_classes = 2
loss_mode = model_name.split('-')[-1].split('.')[0]
if loss_mode.endswith('W'):
    loss_mode = loss_mode[:-1]
print(loss_mode)
crf=False
dist=False
if model_name.split('-')[3]=='crf':
    crf = True
if model_name.split('-')[3]=='dist':
    dist = True

criterion = train.CombinedLoss(loss_mode = loss_mode, weights = None, return_all_losses=True)


In [ ]:


working_dir = os.path.abspath('..')
ckp_file = os.path.join(working_dir, 'models', model_name)

print('Model name:', model_name)
if model_name.startswith('UNetSmall-'):
    model = unets.UNetSmall(in_channels=12, out_channels=num_classes, crf=crf, use_dist=dist).to(device) 
if model_name.startswith('UNet-'):
    model = unets.UNet(in_channels=12, out_channels=num_classes).to(device) 
elif model_name.startswith('UNetResNet34-'):
    model = unets.UNetResNet34(in_channels=12, out_channels=num_classes).to(device) 
elif model_name.startswith('UNetEfficientNetB0-'):
    model = unets.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device) 
elif model_name.startswith('UNetConvNext-'):
    model = unets.UNetConvNext(in_channels=12, out_channels=num_classes).to(device) 
elif model_name.startswith('HRNetW18'):
    model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w18_small", pretrained=True,).to(device)
elif model_name.startswith('HRNetW32'):
    model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w32", pretrained=True,).to(device)
elif model_name.startswith('HRNetW48'):
    model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w48", pretrained=True,).to(device)
checkpoint = torch.load(ckp_file, weights_only=False)
        
start_epoch = checkpoint['epoch'] + 1
best_epoch = checkpoint['best_epoch']
model.load_state_dict(checkpoint['best_model_state_dict'])
#optimizer.load_state_dict(checkpoint['best_optimizer_state_dict']) 
best_val_loss = checkpoint['best_val_loss']
best_epoch_info = checkpoint['best_epoch_info']
current_lr = checkpoint['current_lr']
current_patience = checkpoint['current_parience']       
metadata = checkpoint['metadata']
info = checkpoint['best_epoch_info']
history = checkpoint['history']


In [ ]:
for info in history:
    print(info['train_acc'])
    print(info)
    break


In [ ]:
import matplotlib.pyplot as plt
def plot_col(history, column):

    train_metric = [info[f'train_{column}'] for info in history]
    val_metric = [info[f'val_{column}'] for info in history]
    print(train_metric)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    epochs = range(1, len(train_metric) + 1)

    # Plot training and validation losses
    ax.plot(epochs, train_metric, label=f"Training {column}", marker="o", linestyle="-", color="blue")
    ax.plot(epochs, val_metric, label=f"Validation {column}", marker="o", linestyle="-", color="red")

    # Add labels, title, and legend
    ax.set_title(f"Training and Validation {column}")
    ax.set_xlabel("Epoch")
    ax.set_ylabel(column)
    ax.legend()
    ax.grid(True)


columns = ['loss', 'acc', 'micro', 'macro', 'weighted', 'f1_C0', 'f1_C1', 'f1_C2', 'f1_C3', 'f1_C4', 'CE', 'dice']    
#for c in columns:
#    plot_col(history, c)

def get_best(history):
    lowest_loss = np.inf
    info_best = {}
    for info in history:
        if info['val_loss'] <= lowest_loss:
            lowest_loss = info['val_loss']
            info_best = info
    return info


In [ ]:
columns = ['loss', 'acc', 'micro', 'macro', 'weighted', 'f1_C0', 'f1_C1', 'f1_C2', 'f1_C3', 'f1_C4', 'CE', 'dice']   
for c in columns:
    view.plot_metrics(history, c)


In [ ]:
#Ponderado: nao tao bom pra classe 4, melhor nas 1, 2 e 3.


In [ ]:
tiles = ['032027']#, '032026'] 
num_subtiles = 6
classes_mode = 'type'
training_batch_size = 16
model_types = 'unets'
weighted = True

if classes_mode == 'type':
    num_classes = 5
elif classes_mode == 'density':
    num_classes = 4
elif classes_mode == 'binary':
    num_classes = 2
elif classes_mode == 'all':
    num_classes = 9


working_dir = os.path.abspath('..')
models_paths = os.listdir(os.path.join(working_dir, 'models'))
models_paths = [f for f in models_paths if f.endswith('.pth')]



In [ ]:

train_files, val_files, test_files = data.train_val_test_stratify(tiles, 
                                                                  num_subtiles,
                                                                    train_size = 0.6, 
                                                                    val_size = 0.2, 
                                                                    stratify_by = classes_mode)

In [ ]:
if model_types == 'unets':
    model_param_grid = {
        #model params:
        'model' : [#f'UNetSmall-256-{classes_mode}',
                #f'UNet-256-{classes_mode}', #ok
                #f'UNet-64-{classes_mode}', #ok
                #f'UNetResNet34-224-{classes_mode}', #ok
                #f'UNetEfficientNetB0-224-{classes_mode}', 
                f'UNetConvNext-224-{classes_mode}',
                ],
        
    }

In [ ]:
tile = '032026'

folder = os.path.join(working_dir,f"data/processed/S2-16D_V2_{tile}/{num_subtiles}x{num_subtiles}_subtiles")
files = os.listdir(folder)
files = [os.path.join(folder, f) for f in files if f.endswith('.tif')]





In [ ]:
stride = patch_size-32
for tile_id in ['032026', '025037', '032027']:
    # --------------- opening files -----------------
    folder = os.path.join(working_dir,f"data/processed/S2-16D_V2_{tile_id}/{num_subtiles}x{num_subtiles}_subtiles")
    files = os.listdir(folder)
    files = [os.path.join(folder, f) for f in files if f.endswith('.tif')]
    # --------------- creating a dataloader -----------------
    test_dataset = data.SubtileDataset(files, 
                                    num_subtiles = 6,
                                    classes_mode=classes_mode,
                                    patch_size=patch_size, 
                                    stride=stride, #//2, 
                                    dynamic_sampling = False,
                                    data_augmentation = False, # testando 
                                    return_imgidx = True)
    test_loader = DataLoader(test_dataset, batch_size=training_batch_size, shuffle=False)
    #for image, mask, x, y, f in test_loader:
    #    print(f'{x},{y}',end='|')
    tile = post.ReconstructTile(patch_size = patch_size, stride = stride)

    ### -------------- TESTING -------------------
    import time
    torch.cuda.reset_peak_memory_stats()
    run_time = time.time()
    runner = train.EpochRunner('test', model, test_loader, criterion=criterion, num_classes=num_classes, 
                                optimizer=None, simulated_batch_size = test_loader.batch_size, device = device)  
    for image, label, logits, pred, x, y, f, in runner.run_generator(show_pred = 1):
        tile.add_batch(x, y, f, logits, pred, label, image)
    tile.set_pred()      
    loss, CE, dice, report, acc, cm = runner.get_metrics()
    run_time = time.time()-run_time
    peak_train_memory = f"{torch.cuda.max_memory_allocated() / 1024 ** 2:.2f} MB"
    torch.cuda.empty_cache()

    
    print(f'Test Loss: {loss}, {CE}, {dice}')
    print(f'Test Accuracy: {acc}')
    print(f'Test confusion matrix:')
    view.plot_confusion_matrix(cm)
    print(report)
    
    #### ----------------- Contruct tile
    
    

    r = [0, 10560, 0, 10560]
    #r = [5000, 7000, 1000, 3000]

    plt.figure(figsize=(50, 50))
    plt.subplot(1,2,1)
    plt.imshow(tile.labels[r[0]:r[1], r[2]:r[3]])
    plt.subplot(1,2,2)
    plt.imshow(tile.preds[r[0]:r[1], r[2]:r[3]])
    plt.show()
    break

In [ ]:
print(list(range(4, 16-4)))
plt.figure(figsize=(20,20))
plt.imshow(tile.count[1700-256:1800+256, 1700-256:1800+256])
print(np.unique(tile.count))
print(10560//6)

In [ ]:
print(1504+256)
plt.figure(figsize=(15,15))
plt.imshow(prob[4,230:280, 230:280])

In [ ]:
tile.post_process()

plt.figure(figsize=(50, 50))
plt.subplot(2,2,1)
plt.imshow(tile.cleaned[3000:7000,0:4000])
plt.subplot(2,2,2)
plt.imshow(tile.crf[3000:7000,0:4000])
plt.subplot(2,2,3)
plt.imshow(tile.cleaned_crf[3000:7000,0:4000])
plt.subplot(2,2,4)
plt.imshow(tile.vi_completion[3000:7000,0:4000])
plt.show()





In [ ]:
from skimage.morphology import remove_small_objects

def remove_small(final_mask, min_size = 10):
    final_mask = remove_small_objects(final_mask, min_size=min_size)
    return final_mask

a = remove_small(tile.crf, min_size = 25)
plt.imshow(a)

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
t = tile.tile
t=t-np.min(t)
t/=np.max(t)

r = [3000, 7000, 0, 4000]
#plt.imshow(t, cmap='gray')
plt.figure(figsize=(50, 50))
plt.subplot(1,2,1)
plt.imshow(tile.labels[r[0]:r[1],r[2]:r[3]])
plt.subplot(1,2,2)
plt.imshow(tile.preds[r[0]:r[1],r[2]:r[3]])
#plt.subplot(1,3,3)
#plt.imshow(tile_032026.cleaned_preds[3000:7000,0:4000])
plt.show()

In [ ]:
view.plt_tile(tile.labels[3000:7000,0:4000], tile.preds[3000:7000,0:4000])


In [ ]:
label_map = {
        0: ('Reds', 0.0, 'Fundo'),
        1: ('Blues', 0.8, 'Loteamento Vazio'),
        2: ('Greens', 0.8, 'Outros Equipamentos'),
        3: ('Reds', 0.8, 'Vazio Intraurbano'),
        4: ('Oranges', 0.8, 'Área Urbanizada')
    }
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
view.plot_masked_image(tile.labels[3000:7000,0:4000], label_map, image=None, title="Mask Overlay Only", ax=axs[0])
view.plot_masked_image(tile.preds[3000:7000,0:4000], label_map, image=None, title="Mask Overlay Only", ax=axs[1])
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(2, 2, figsize=(16, 16))
view.plot_masked_image(tile.cleaned_preds[3000:7000,0:4000], label_map, image=None, title="Cleaned", ax=axs[0])
view.plot_masked_image(tile.crf[3000:7000,0:4000], label_map, image=None, title="CRF", ax=axs[1])
view.plot_masked_image(tile.cleaned_crf[3000:7000,0:4000], label_map, image=None, title="CRF+Clean", ax=axs[2])
view.plot_masked_image(tile.vi_completion[3000:7000,0:4000], label_map, image=None, title="+completion", ax=axs[3])
plt.tight_layout()
plt.show()


In [ ]:
pca_config_file = os.path.join(working_dir, 'config', 'pca_weights.npy')
print(tile.image.shape)
pca_img = torch.Tensor(data.apply_pca_weights(tile.image, pca_config_file))
pca_img = pca_img.permute(1, 2, 0).detach().cpu().numpy()
#pca_img -= np.min(pca_img)
#pca_img /= np.max(pca_img)

label_map = {
        0: ('Reds', 0.1, 'Fundo'),
        1: ('Blues', 0.8, 'Loteamento Vazio'),
        2: ('Greens', 0.8, 'Outros Equipamentos'),
        3: ('Purples', 0.8, 'Vazio Intraurbano'),
        4: ('Oranges', 0.8, 'Área Urbanizada')
    }
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
view.plot_masked_image(tile.labels[3000:7000,0:4000], label_map, image=pca_img[3000:7000,0:4000], title="Mask Overlay Only", ax=axs[0])
view.plot_masked_image(tile.preds[3000:7000,0:4000], label_map, image=pca_img[3000:7000,0:4000], title="Mask Overlay Only", ax=axs[1])
plt.tight_layout()
plt.show()

In [ ]:
np.max(pca_img)


In [ ]:
#plt.figure(figsize=(10,10))
#plt.imshow(tile_032026.logits)
tile_032026.preds[:200,:200]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(tile_032026.logits_0)

In [ ]:
x = (256, 256+32)
y = (2256, 2256+32)

plt.figure(figsize=(10,10))
plt.imshow(tile_032026.logits[x[0]:x[1],y[0]:y[1]])
plt.figure(figsize=(10,10))
plt.imshow(tile_032026.logits_0[x[0]:x[1],y[0]:y[1]])
plt.figure(figsize=(10,10))
plt.imshow(tile_032026.labels[x[0]:x[1],y[0]:y[1]])

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))
sigmoid(tile_032026.logits_0[:10,:10])
plt.imshow(sigmoid(tile_032026.logits[x[0]:x[1],y[0]:y[1]]))

In [ ]:
import os
import pandas as pd

# Define the folder containing the CSV files
working_dir = os.path.abspath('..')
folder_path = os.path.join(working_dir, 'experimental_results')

# Initialize an empty list to store the rows
rows_list = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Filter rows where the 'Status' column contains 'current best val loss.'
        filtered_df = df[df['Status'].str.contains('current best val loss.', na=False)]
        
        # If there are any matching rows, take the last one
        if not filtered_df.empty:
            last_row = filtered_df.iloc[-1]
            rows_list.append(last_row)

# Combine all the rows into a single DataFrame
combined_df = pd.DataFrame(rows_list)

# Reset the index of the combined DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
combined_df

In [ ]:
combined_df['Model path'][3]

In [ ]:
combined_df[combined_df['Model']=='UNetSmall-256-type-diceW']

In [ ]:
num_iter=100
start_lr=1e-6
end_lr=1e-3
lr_mult = (end_lr / start_lr) ** (1/num_iter)
lr_mult

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_lr_finder import LRFinder

# Dummy dataset
class DummyDataset(Dataset):
    def __len__(self):
        return 100
    def __getitem__(self, idx):
        x = torch.randn(3, 32, 32)  # Input: 3x32x32 image
        y = torch.randint(0, 10, (1,)).item()  # Target: class label
        return x, y  # Must return (input, target)

# Dummy model
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(3*32*32, 10)
).cuda()

# DataLoader
train_loader = DataLoader(DummyDataset(), batch_size=8)

# Test LRFinder
optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)
lr_finder = LRFinder(model, optimizer, torch.nn.CrossEntropyLoss())
lr_finder.range_test(train_loader, end_lr=0.1, num_iter=100)  # Should work!

In [ ]:
for x, y in train_loader:
    print(x, y)
    break

In [ ]:
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

class DistLayer(torch.nn.Linear):
    def __init__(self, in_features, out_features, n=1., eps=1e-6, bias=False):
        super(DistLayer, self).__init__(in_features, out_features, bias=bias)
        self.n = n
        self.eps = eps
        
    def forward(self, x, scale=False):
        # x: (B, N)
        # w: (V, N)
        # dist_sq: (B, V)
        n_embd = x.size(-1,)
        w = self.weight
        #w.data *= 0.
        wx = torch.einsum('bn,vn->bv', x, w) # (B, V)
        ww = torch.norm(w, dim=-1)**2 # (V,)
        xx = torch.norm(x, dim=-1)**2 # (B,)

        dist_sq = ww[None,:] + xx[:,None] - 2 * wx + self.eps
        dist_sq = dist_sq / torch.min(dist_sq, dim=-1, keepdim = True)[0]
        return (dist_sq)**(-self.n)

inputs = torch.tensor([[0.,1.], [0., -1,], [-1., 0.], [1., 0.], [0., 0.]])
labels = torch.tensor([0,1,2,3,4], dtype=torch.long)

distlayer = DistLayer(2,5)

lr = 1e-2
n_steps = 10000
optimizer = torch.optim.Adam(distlayer.parameters(), lr = lr)
ws = []
w_l2s = []
ls = []

for i in range(n_steps):
    optimizer.zero_grad()
    
    prob = distlayer(inputs)
    prob = prob/torch.sum(prob, dim=1, keepdim=True)
    logits = torch.log(prob)
    loss = torch.nn.functional.cross_entropy(logits, labels)
    loss.backward()
    optimizer.step()
    
    ws.append(copy.deepcopy(distlayer.weight.detach().numpy()))
    w_l2s.append(np.linalg.norm(distlayer.weight.detach().numpy()))
    ls.append(loss.item())
    
ls_h = np.array(ls)
w_l2s_h = np.array(w_l2s)

In [ ]:
inputs.shape
logits

In [ ]:
distlayer.weight